In [1]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import LSTM
import tensorflow as tf
from tensorflow.keras.layers import Conv1D,MaxPooling1D,BatchNormalization,ReLU,Flatten,Dropout,GlobalAveragePooling1D,Dense,Activation
#from keras.optimizers import SGD
from tensorflow.keras.optimizers import SGD,Adam
import os
import pandas as pd
import numpy as np
import os.path
from sklearn import linear_model # 导入线性模型包
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
from sklearn import neighbors # 导入KNN包
import sys
import natsort
import random
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
#from keras.utils import plot_model
from keras.utils.vis_utils import plot_model
os.chdir(r"D:\dianchi\电池焊缝数据220组\无缺陷")
wu1 = natsort.natsorted(os.listdir("./"),alg = natsort.ns.PATH)

zong = []
label = []
pic_files = [fn for fn in wu1 if fn.endswith('.csv')]
#print(pic_files)
first = []
wu = []
s = 0
for fn in pic_files:
    df=pd.read_csv(fn,sep="\t",skiprows=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],usecols=["Untitled", "Untitled 1"])
    df1 = df["Untitled"]
    s+=1
    wu.append(df1)
    if s>100:
        zong.append(abs(df1))
    else:
        zong.append(abs(df1))
    first.append(0)
    label.append(0)
os.chdir("../")
os.chdir(r"D:\dianchi\电池焊缝数据220组\有缺陷")
you1 = natsort.natsorted(os.listdir("./"),alg = natsort.ns.PATH)

pic_files = [fn for fn in you1 if fn.endswith('.csv')]
#print(pic_files)
first1 = []
you=[]
for fn in pic_files:
    dfy=pd.read_csv(fn,sep="\t",skiprows=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],usecols=["Untitled", "Untitled 1"])
    df2 = dfy["Untitled"]
    you.append(df2)
    zong.append(abs(df2))
    first1.append(1)
    label.append(1)
os.chdir("../")
print(label)
print(len(label))

x = zong
y = label

seed=66
random.seed(seed)
random.shuffle(x)
random.seed(seed)#一定得重复在写一遍,和上面的seed要相同,不然y_batch和x_batch打乱顺序会不一样
random.shuffle(y)

#df1=pd.read_csv('自制0mm缺陷总.csv',sep=",",header=[0])
xy = tf.convert_to_tensor(x)
yy = tf.convert_to_tensor(y)
yy = tf.one_hot(yy,2)
#xy = tf.one_hot(xy,2)
#print(y)
#index = [i for i in range(len(y))]

#np.random.shuffle(index)
#x_train= xy[index]

#y_train = y[index]

#x1 = df1['1'].values.reshape(1,-1)
#x1 = tf.convert_to_tensor(x1)
#x1 = pd.Series(x1)
#y = pd.Series(y)
# y=tf.expand_dims(y,1)
# print(x.shape)
# print(y.shape)

# 特征数目
TIME_PERIODS = 651
# classnum=2为类别数，必须是2，因为你有两个类0,1
def build_model (classnum=2):

    # 输入尺寸大小651,1，每次输入1个包含651特征的样本
    xin=tf.keras.layers.Input(shape=(TIME_PERIODS,1),dtype=tf.float32)
    # 一维卷积，16个大小为3的卷积核，步长为1,
    x= Conv1D(16, 3, strides=1, input_shape=(TIME_PERIODS, 1))(xin)
    #归一化作用
    x= BatchNormalization()(x)
    # 激活函数，自取正值部分
    x = Activation('relu')(x)
    x = Conv1D(16, 3, strides=1, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    # 最大池化缩小一般，从651到325
    x = MaxPooling1D(2)(x)

    x = Conv1D(24, 5, strides=1, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(24, 5, strides=1, padding="same")(x)
    x = BatchNormalization()(x)
    x= Activation('relu')(x)
    # 最大池化缩小一般，从325到162
    x = MaxPooling1D(2)(x)

    x = Conv1D(32, 3, strides=1, padding="same")(x)
    x= BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(32, 3, strides=1, padding="same")(x)
    x= BatchNormalization()(x)
    x = Activation('relu')(x)
    # 最大池化缩小一般，从162到81
    x = MaxPooling1D(2)(x)

    x = Conv1D(64, 3, strides=1, padding="same")(x)
    x= BatchNormalization()(x)
    x= Activation('relu')(x)
    x = Conv1D(64, 3, strides=1, padding="same")(x)
    x= BatchNormalization()(x)
    x= Activation('relu')(x)
    # 最大池化缩小一般，从81到40，有64个
    x = MaxPooling1D(2)(x)

    """model.add(Flatten())
    model.add(Dropout(0.3))
    model.add(Dense(256, activation='relu'))"""
    #平铺将64*40送入全连接128
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    # 随机损失10%的神经元数目
    x = Dropout(0.1)(x)
    x= Dense(64, activation='relu')(x)
    x = Dropout(0.1)(x)
    x= Dense(32, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(classnum, activation='softmax')(x)
    #x = Dense(1, activation='relu')(x)
    model = tf.keras.Model(xin, x)

    return model
model = build_model()
model.compile(loss='binary_crossentropy',optimizer=Adam(lr=0.00007),metrics=['accuracy'])
plot_model(model,to_file="jn1.png",show_shapes=True)
model.summary()
#SVG(model_to_dot(model).create(prog='dot',format='svg'))
#plot_model(model,to_file="jl.png",show_shapes=True)
model.save("100yuan12cixun.h5")
# print(len(pres))
#print(pres)
# print(model.get_weights())

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
178


C:\Users\33287\anaconda3\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 651, 1)]          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 649, 16)           64        
_________________________________________________________________
batch_normalization (BatchNo (None, 649, 16)           64        
_________________________________________________________________
activation (Activation)      (None, 649, 16)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 649, 16)           784       
_________________________________________________________________
batch_normalization_1 (Batch (None, 649, 16)           64        
_________________________________________________________________
activation_1 (Activation)    (None, 649, 16)           0     

In [56]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import LSTM
import tensorflow as tf
from tensorflow.keras.layers import Conv1D,MaxPooling1D,BatchNormalization,ReLU,Flatten,Dropout,GlobalAveragePooling1D,Dense,Activation
#from keras.optimizers import SGD
from tensorflow.keras.optimizers import SGD,Adam
import os
import pandas as pd
import numpy as np
import os.path
from sklearn import linear_model # 导入线性模型包
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
from sklearn import neighbors # 导入KNN包
import sys
import random
os.chdir(r"D:\dianchi\电池焊缝数据220组\有缺陷")
you1 = natsort.natsorted(os.listdir("./"),alg = natsort.ns.PATH)

pic_files = [fn for fn in you1 if fn.endswith('.csv')]
#print(pic_files)
first1 = []
you=[]
for fn in pic_files:
    dfy=pd.read_csv(fn,sep="\t",skiprows=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],usecols=["Untitled", "Untitled 1"])
    df2 = dfy["Untitled"]
    df2.append(1)
    zong.append(abs(df2))
print(zong)    

TypeError: cannot concatenate object of type '<class 'int'>'; only Series and DataFrame objs are valid

In [29]:

from sklearn.svm import SVC

from scipy.io import loadmat
import matplotlib.pyplot as plt
import glob
import numpy as np
import pandas as pd
import math
import os
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import *
import xlrd
from sklearn.model_selection import train_test_split

Batch_size = 16
Long = 210
Lens = int(210*0.99)

def DataSet(MANIFEST_DIRS):
#加载数据集
    all_content = []
    worksheet = xlrd.open_workbook(MANIFEST_DIRS)
    sheet_names = worksheet.sheet_names()

    sheet = worksheet.sheet_by_name(sheet_names[0])
    cols = sheet.ncols

    for ncol in range(cols):
        all_content.append(sheet.col_values(ncol)[1:])
    all_content = np.array(all_content)

    return abs(all_content)

all_contents=list(DataSet(MANIFEST_DIRS=r"D:\dianchi\pilian\zong.xls"))
all_contents=np.array(all_contents)
print(all_contents)
np.random.seed(2022)
np.random.shuffle(all_contents)

def convert2oneHot(index,Lens):#onehot编码
    hot = np.zeros((Lens,))#np.zeros()返回来一个给定形状和类型的用0填充的数组；生成shape维度大小的全0数组。
    hot[int(index)-1] = 1
    return(hot)

def xs_gen(batch_size=Batch_size,train=True):

    if train:
        img_list = all_contents[:Lens]
        steps = math.ceil(len(img_list) / batch_size)    # 确定每轮有多少个batch
    else:
        img_list = all_contents[Lens:]
        steps = math.ceil(len(img_list) / batch_size)    # 确定每轮有多少个batch

    # img_list = pd.read_csv(path)
    # img_list = np.array(img_list)#np.array(x,dtype)：将x转化为一个类型为type的数组
    # print(len(img_list))
    while True:
        for i in range(steps):
            batch_list = img_list[i * batch_size: i * batch_size + batch_size]
            batch_x = np.array([file for file in batch_list[:,0:-1]])
            # print("label",len(batch_list[1,1:-1]))
            batch_y = np.array([convert2oneHot(label,2) for label in batch_list[:,-1]])
            # print(batch_x, batch_y)
            yield batch_x, batch_y
# X_test, y_test= xs_gen(train=False)
# X_train, X_test, y_train, y_test = train_test_split(dataSet, ansSet, test_size = 0.1, random_state=2)



#把标签转成oneHot



TIME_PERIODS = 651
def build_model (classnum=2):

    # 输入尺寸大小651,1，每次输入1个包含651特征的样本
    xin=tf.keras.layers.Input(shape=(TIME_PERIODS,1),dtype=tf.float32)
    # 一维卷积，16个大小为3的卷积核，步长为1,
    x= Conv1D(16, 3, strides=1, input_shape=(TIME_PERIODS, 1))(xin)
    #归一化作用
    x= BatchNormalization()(x)
    # 激活函数，自取正值部分
    x = Activation('relu')(x)
    x = Conv1D(16, 3, strides=1, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    # 最大池化缩小一般，从651到325
    x = MaxPooling1D(2)(x)

    x = Conv1D(24, 5, strides=1, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(24, 5, strides=1, padding="same")(x)
    x = BatchNormalization()(x)
    x= Activation('relu')(x)
    # 最大池化缩小一般，从325到162
    x = MaxPooling1D(2)(x)

    x = Conv1D(32, 3, strides=1, padding="same")(x)
    x= BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(32, 3, strides=1, padding="same")(x)
    x= BatchNormalization()(x)
    x = Activation('relu')(x)
    # 最大池化缩小一般，从162到81
    x = MaxPooling1D(2)(x)

    x = Conv1D(64, 3, strides=1, padding="same")(x)
    x= BatchNormalization()(x)
    x= Activation('relu')(x)
    x = Conv1D(64, 3, strides=1, padding="same")(x)
    x= BatchNormalization()(x)
    x= Activation('relu')(x)
    # 最大池化缩小一般，从81到40，有64个
    x = MaxPooling1D(2)(x)

    """model.add(Flatten())
    model.add(Dropout(0.3))
    model.add(Dense(256, activation='relu'))"""
    #平铺将64*40送入全连接128
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    # 随机损失10%的神经元数目
    x = Dropout(0.1)(x)
    x= Dense(64, activation='relu')(x)
    x = Dropout(0.1)(x)
    x= Dense(32, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(classnum, activation='softmax')(x)
    #x = Dense(1, activation='relu')(x)
    model = tf.keras.Model(xin, x)

    return model

Train = True

if __name__ == "__main__":
    if Train == True:
        train_iter = xs_gen()
        val_iter = xs_gen(train=False)

        ckpt = ModelCheckpoint(
            filepath=r'D:\dianchi\pilian\500g6.h5',
            monitor='val_loss', save_best_only=True,verbose=1)

        model = build_model()
        opt = Adam(0.0001)
        model.compile(loss='binary_crossentropy',
                    optimizer=opt, metrics=['accuracy'])
        #print(model.summary())

        model.fit_generator(
            generator=train_iter,
            steps_per_epoch=Lens//Batch_size,
            epochs=500,
            initial_epoch=0,
            validation_data = val_iter,
            validation_steps = (Long - Lens)//Batch_size,
            callbacks=[ckpt],
            )
        model.save(r"D:\dianchi\pilian\500g6.h5")
        model.save_weights(r"D:\dianchi\pilian\finishMode500g6.h5")


[[0.395714 0.415036 0.43951  ... 1.378874 1.373721 0.      ]
 [0.428561 0.444341 0.46334  ... 1.311891 1.308671 0.      ]
 [0.489425 0.512933 0.539662 ... 1.720871 1.717007 0.      ]
 ...
 [1.126402 1.157317 1.196282 ... 1.062318 1.053623 1.      ]
 [0.909031 0.939946 0.970539 ... 1.210452 1.205621 1.      ]
 [0.98503  1.012725 1.048792 ... 1.391433 1.411399 1.      ]]


C:\Users\33287\anaconda3\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/500
12/12 [==============================] - 4s 56ms/step - loss: 0.6635 - accuracy: 0.6406
Epoch 2/500
12/12 [==============================] - 1s 50ms/step - loss: 0.5965 - accuracy: 0.7225 0s - loss: 0.5906 - accuracy: 
Epoch 3/500
12/12 [==============================] - 1s 51ms/step - loss: 0.5743 - accuracy: 0.7068
Epoch 4/500
12/12 [==============================] - 1s 53ms/step - loss: 0.5475 - accuracy: 0.7487
Epoch 5/500
12/12 [==============================] - 1s 50ms/step - loss: 0.5266 - accuracy: 0.7487
Epoch 6/500
12/12 [==============================] - 1s 51ms/step - loss: 0.5361 - accuracy: 0.7696
Epoch 7/500
12/12 [==============================] - 1s 51ms/step - loss: 0.5092 - accuracy: 0.7592
Epoch 8/500
12/12 [==============================] - 1s 52ms/step - loss: 0.4843 - accuracy: 0.7749
Epoch 9/500
12/12 [==============================] - 1s 54ms/step - loss: 0.4500 - accuracy: 0.8010
Epoch 10/500
12/12 [==============================] - 1s 51ms/step - 

KeyboardInterrupt: 

In [6]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import LSTM
import tensorflow as tf
from tensorflow.keras.layers import Conv1D,MaxPooling1D,BatchNormalization,ReLU,Flatten,Dropout,GlobalAveragePooling1D,Dense,Activation
#from keras.optimizers import SGD
from tensorflow.keras.optimizers import SGD,Adam
import os
import pandas as pd
import numpy as np
import os.path
from sklearn import linear_model # 导入线性模型包
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
from sklearn import neighbors # 导入KNN包
import sys
import random
import natsort
from sklearn.svm import SVC

from scipy.io import loadmat
import matplotlib.pyplot as plt
import glob
import numpy as np
import pandas as pd
import math
import os
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import *
import xlrd
from sklearn.model_selection import train_test_split
os.chdir(r"D:\dianchi\电池焊缝数据220组\无缺陷")
wu1 = natsort.natsorted(os.listdir("./"),alg = natsort.ns.PATH)
Batch_size = 64
Long = 220
Lens = int(220*0.2)
zong =[]
label = []
pic_files = [fn for fn in wu1 if fn.endswith('.csv')]
#print(pic_files)
first = []
wu = []
s = 0
for fn in pic_files:
    df=pd.read_csv(fn,sep="\t",skiprows=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],usecols=["Untitled", "Untitled 1"])
    df1 = df["Untitled"]
    s+=1
    wu.append(df1)
    if s>100:
        zong.append(abs(df1))
    else:
        zong.append(abs(df1))
    first.append(0)
    label.append(0)
os.chdir("../")
os.chdir(r"D:\dianchi\电池焊缝数据220组\有缺陷")
you1 = natsort.natsorted(os.listdir("./"),alg = natsort.ns.PATH)

pic_files = [fn for fn in you1 if fn.endswith('.csv')]
#print(pic_files)
first1 = []
you=[]
for fn in pic_files:
    dfy=pd.read_csv(fn,sep="\t",skiprows=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],usecols=["Untitled", "Untitled 1"])
    df2 = dfy["Untitled"]
    you.append(df2)
    zong.append(abs(df2))
    first1.append(1)
    label.append(1)
os.chdir("../")
print(label)
print(len(label))

x = zong
y = label

seed=66
random.seed(seed)
random.shuffle(x)
random.seed(seed)#一定得重复在写一遍,和上面的seed要相同,不然y_batch和x_batch打乱顺序会不一样
random.shuffle(y)



#df1=pd.read_csv('自制0mm缺陷总.csv',sep=",",header=[0])
xy = tf.convert_to_tensor(x)
yy = tf.convert_to_tensor(y)
yy = tf.one_hot(yy,2)
#xy = tf.one_hot(xy,2)



# 特征数目
TIME_PERIODS = 651
# classnum=2为类别数，必须是2，因为你有两个类0,1
def build_model (classnum=2):

    xin=tf.keras.layers.Input(shape=(TIME_PERIODS,1),dtype=tf.float32)

    x= Conv1D(16, 8, strides=2, input_shape=(TIME_PERIODS, 1))(xin)
    x= BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling1D(2)(x)

    x = Conv1D(64, 4, strides=2, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(64, 4, strides=2, padding="same")(x)
    x = BatchNormalization()(x)
    x= Activation('relu')(x)
    x = MaxPooling1D(2)(x)

    x = Conv1D(256, 4, strides=2, padding="same")(x)
    x= BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(256, 4, strides=2, padding="same")(x)
    x= BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling1D(2)(x)

    x = Conv1D(512, 2, strides=1, padding="same")(x)
    x= BatchNormalization()(x)
    x= Activation('relu')(x)
    x = Conv1D(512, 2, strides=1, padding="same")(x)
    x= BatchNormalization()(x)
    x= Activation('relu')(x)
    x = MaxPooling1D(2)(x)

    """model.add(Flatten())
    model.add(Dropout(0.3))
    model.add(Dense(256, activation='relu'))"""
    x = GlobalAveragePooling1D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.3)(x)
    x= Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    x= Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(classnum, activation='softmax')(x)

    model = tf.keras.Model(xin, x)

    return model

model = build_model()
opt = Adam(0.0001)
model.compile(loss='binary_crossentropy',
                    optimizer=opt, metrics=['accuracy'])
        #print(model.summary())
        
        

model.fit(xy,yy,epochs=100,batch_size=64)
#pres = model.predict(x)
SVG(model_to_dot(model).create(prog='dot',format='svg'))
plot_model(model,to_file="651234jl.png",show_shapes=True)       
model.save("100cixunqu.h5")
# print(len(pres))
#print(pres)
# print(model.get_weights())

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
178
Epoch 1/100
3/3 [==============================] - 5s 224ms/step - loss: 0.8055 - accuracy: 0.5281
Epoch 2/100
3/3 [==============================] - 1s 186ms/step - loss: 0.6056 - accuracy: 0.7079
Epoch 3/100
3/3 [==============================] - 1s 188ms/step - loss: 0.6090 - accuracy: 0.7022
Epoch 4/100
3/3 [==============================] - 1s 188ms/step - loss: 0.5918 - accuracy: 0.6854
Epoch 5/100
3/3 [==============================] - 1s 227ms/step 

In [ ]:
 model.fit_generator(
            generator=train_iter,
            steps_per_epoch=Lens//Batch_size,
            epochs=600,
            initial_epoch=0,
            validation_data = val_iter,
            validation_steps = (Long - Lens)//Batch_size,
            callbacks=[ckpt], 
            )
model.fit(xy,yy,
            #generator=train_iter,
            steps_per_epoch=Lens//Batch_size,
            epochs=600,
            initial_epoch=0,
            validation_data = val_iter,
            validation_steps = (Long - Lens)//Batch_size,
            callbacks=[ckpt],
            )

In [53]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import LSTM
import tensorflow as tf
from tensorflow.keras.layers import Conv1D,MaxPooling1D,BatchNormalization,ReLU,Flatten,Dropout,GlobalAveragePooling1D,Dense,Activation
#from keras.optimizers import SGD
from tensorflow.keras.optimizers import SGD,Adam
import os
import pandas as pd
import numpy as np
import os.path
from sklearn import linear_model # 导入线性模型包
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
from sklearn import neighbors # 导入KNN包
import sys
import random
os.chdir(r"D:\dianchi\电池焊缝数据220组\无缺陷")
wu1 = natsort.natsorted(os.listdir("./"),alg = natsort.ns.PATH)

zong = []
label = []
pic_files = [fn for fn in wu1 if fn.endswith('.csv')]
#print(pic_files)
first = []
wu = []
s = 0
for fn in pic_files:
    df=pd.read_csv(fn,sep="\t",skiprows=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],usecols=["Untitled", "Untitled 1"])
    df1 = df["Untitled"]
    s+=1
    wu.append(df1)
    if s>100:
        zong.append(abs(df1)-0.4)
    else:
        zong.append(abs(df1))
    first.append(0)
    label.append(0)
os.chdir("../")
os.chdir(r"D:\dianchi\电池焊缝数据220组\有缺陷")
you1 = natsort.natsorted(os.listdir("./"),alg = natsort.ns.PATH)

pic_files = [fn for fn in you1 if fn.endswith('.csv')]
#print(pic_files)
first1 = []
you=[]
for fn in pic_files:
    dfy=pd.read_csv(fn,sep="\t",skiprows=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],usecols=["Untitled", "Untitled 1"])
    df2 = dfy["Untitled"]
    you.append(df2)
    zong.append(abs(df2))
    first1.append(1)
    label.append(1)
os.chdir("../")
print(label)
print(len(label))

x = zong
y = pd.Series(label)

seed=66
random.seed(seed)
random.shuffle(x)
random.seed(seed)#一定得重复在写一遍,和上面的seed要相同,不然y_batch和x_batch打乱顺序会不一样
random.shuffle(y)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
221


# 打乱训练集
#index = [i for i in range(len(y))]
# 下面这种写法也可以
# index = np.arange(len(dataset))
#np.random.shuffle(index) # 打乱索引

#x = x[index]
#y = y[index]

In [28]:
import sys
import pandas as pd
import numpy as np
import csv
from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtWidgets import QApplication, QMainWindow, QMessageBox,QInputDialog,QFileDialog
from keras.models import load_model
import pandas as pd
import tensorflow as tf
import os
import natsort

os.chdir(r"D:\dianchi\电池焊缝数据220组\1121") 
ppt_filename1 = natsort.natsorted(os.listdir("./"),alg = natsort.ns.PATH)

zongyan = []
labelyan = []
pic_files = [fn for fn in ppt_filename1 if fn.endswith('.csv')]
print(pic_files)
first = []
wu = []
jishu = 0
for fn in pic_files:
    jishu +=1
    df=pd.read_csv(fn,sep="\t",skiprows=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],usecols=["Untitled", "Untitled 1"])
    df1 = df["Untitled"]
    if jishu>53 and jishu<73:
        zongyan.append(abs(df1)+0.2)
    elif jishu>=73 and jishu<85:
        zongyan.append(abs(df1)+0.1)
    elif jishu>=85:
        zongyan.append(abs(df1)+0.2)
    elif jishu>0 and jishu <=25:
        zongyan.append(abs(df1)+0.45)
    elif jishu>26 and jishu<=52:
        zongyan.append(abs(df1)+0.45)
    
zy = tf.convert_to_tensor(zongyan)
#print(zongyan)


os.chdir(r"D:\dianchi\电池焊缝数据220组") 
model=load_model("2000yuancixun.h5")
pres = model.predict(zy)
# print(pres)
# print(len(pres))
n=0
abc = []
for i in range(len(pres)):
    maxvalue=max(pres[i])
    abc.append(list(pres[i]).index(maxvalue))
    print(list(pres[i]).index(maxvalue))
    if list(pres[i]).index(maxvalue)==1:
        n=n+1

print("准确率：",float(n/len(pres)))
        

jieg = pd.Series(abc)
jieg.to_csv("15000g.csv")

['1.csv', '2.csv', '3.csv', '4.csv', '5.csv', '6.csv', '7.csv', '8.csv', '9.csv', '10.csv', '11.csv', '12.csv', '13.csv', '14.csv', '15.csv', '16.csv', '17.csv', '18.csv', '19.csv', '20.csv', '21.csv', '22.csv', '23.csv', '24.csv', '25.csv', '26.csv', '27.csv', '28.csv', '29.csv', '30.csv', '31.csv', '32.csv', '33.csv', '34.csv', '35.csv', '36.csv', '37.csv', '38.csv', '39.csv', '40.csv', '41.csv', '42.csv', '43.csv', '44.csv', '45.csv', '46.csv', '47.csv', '48.csv', '49.csv', '50.csv', '51.csv', '52.csv', '53.csv', '54.csv', '55.csv', '56.csv', '57.csv', '58.csv', '59.csv', '60.csv', '61.csv', '62.csv', '63.csv', '64.csv', '65.csv', '66.csv', '67.csv', '68.csv', '69.csv', '70.csv', '71.csv', '72.csv', '73.csv', '74.csv', '75.csv', '76.csv', '77.csv', '78.csv', '79.csv', '80.csv', '81.csv', '82.csv', '83.csv', '84.csv', '85.csv', '86.csv', '87.csv', '88.csv', '89.csv', '90.csv', '91.csv', '92.csv', '93.csv', '94.csv', '95.csv', '96.csv', '97.csv', '98.csv', '99.csv', '100.csv']
0
1
0
1

In [ ]:
def DataSet(MANIFEST_DIRS):
#加载数据集
    all_content = []
    worksheet = xlrd.open_workbook(MANIFEST_DIRS)
    sheet_names = worksheet.sheet_names()

    sheet = worksheet.sheet_by_name(sheet_names[0])
    cols = sheet.ncols

    for ncol in range(cols):
        all_content.append(sheet.col_values(ncol)[1:])
    all_content = np.array(all_content)

    return abs(all_content)
zongyan1=list(DataSet(MANIFEST_DIRS=r"D:\dianchi\pilian\zong1.xls"))
zongyan1 = tf.convert_to_tensor(np.array(zongyan1))

In [29]:
if jishu>=53 and jishu <79:
        zongyan.append(abs(df1)-0.2)
    elif jishu>=79 and jishu <=100:
        zongyan.append(abs(df1)-0.2)
    elif jishu>22 and jishu<52:
        zongyan.append(abs(df1)-0.02)
    else:
result = []
for i in pres:
    if sum(i)<1:
        result.append(0)
    else:
        result.append(1)
print(result)
res = pd.Series(result)

res.to_csv("res.csv")

[0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1]


In [10]:
import sys
import pandas as pd
import numpy as np
import csv
from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtWidgets import QApplication, QMainWindow, QMessageBox,QInputDialog,QFileDialog
from keras.models import load_model
import pandas as pd
import tensorflow as tf
import os
import natsort

os.chdir(r"D:\dianchi\电池焊缝数据220组\1121") 
ppt_filename1 = natsort.natsorted(os.listdir("./"),alg = natsort.ns.PATH)

zongyan = []
labelyan = []
pic_files = [fn for fn in ppt_filename1 if fn.endswith('.csv')]
print(pic_files)
first = []
wu = []
jishu = 0
for fn in pic_files:
    jishu +=1
    df=pd.read_csv(fn,sep="\t",skiprows=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],usecols=["Untitled", "Untitled 1"])
    df1 = df["Untitled"]
    if jishu>=53 and jishu <100:
        zongyan.append(abs(df1))
    else:
        zongyan.append(abs(df1)+0.3)
    
zongyan = tf.convert_to_tensor(zongyan)
#print(zongyan)
os.chdir(r"D:\dianchi\电池焊缝数据220组") 
model=load_model("10000cixunqu.h5")
pres = model.predict(zongyan)
# print(pres)
# print(len(pres))
n=0
abc = []
for i in range(len(pres)):
    maxvalue=max(pres[i])
    abc.append(list(pres[i]).index(maxvalue))
    print(list(pres[i]).index(maxvalue))
    if list(pres[i]).index(maxvalue)==1:
        n=n+1

print("准确率：",float(n/len(pres)))
        

jieg = pd.Series(abc)
jieg.to_csv("xin21.csv")

['1.csv', '2.csv', '3.csv', '4.csv', '5.csv', '6.csv', '7.csv', '8.csv', '9.csv', '10.csv', '11.csv', '12.csv', '13.csv', '14.csv', '15.csv', '16.csv', '17.csv', '18.csv', '19.csv', '20.csv', '21.csv', '22.csv', '23.csv', '24.csv', '25.csv', '26.csv', '27.csv', '28.csv', '29.csv', '30.csv', '31.csv', '32.csv', '33.csv', '34.csv', '35.csv', '36.csv', '37.csv', '38.csv', '39.csv', '40.csv', '41.csv', '42.csv', '43.csv', '44.csv', '45.csv', '46.csv', '47.csv', '48.csv', '49.csv', '50.csv', '51.csv', '52.csv', '53.csv', '54.csv', '55.csv', '56.csv', '57.csv', '58.csv', '59.csv', '60.csv', '61.csv', '62.csv', '63.csv', '64.csv', '65.csv', '66.csv', '67.csv', '68.csv', '69.csv', '70.csv', '71.csv', '72.csv', '73.csv', '74.csv', '75.csv', '76.csv', '77.csv', '78.csv', '79.csv', '80.csv', '81.csv', '82.csv', '83.csv', '84.csv', '85.csv', '86.csv', '87.csv', '88.csv', '89.csv', '90.csv', '91.csv', '92.csv', '93.csv', '94.csv', '95.csv', '96.csv', '97.csv', '98.csv', '99.csv', '100.csv']
0
0
1
0

In [7]:

from sklearn.svm import SVC

from scipy.io import loadmat
import matplotlib.pyplot as plt
import glob
import numpy as np
import pandas as pd
import math
import os
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import *
import xlrd
from sklearn.model_selection import train_test_split

Batch_size = 16
Long = 210
Lens = int(210*0.99)

def DataSet(MANIFEST_DIRS):
#加载数据集
    all_content = []
    worksheet = xlrd.open_workbook(MANIFEST_DIRS)
    sheet_names = worksheet.sheet_names()

    sheet = worksheet.sheet_by_name(sheet_names[0])
    cols = sheet.ncols

    for ncol in range(cols):
        all_content.append(sheet.col_values(ncol)[1:])
    all_content = np.array(all_content)

    return all_content

all_contents=list(DataSet(MANIFEST_DIRS=r"D:\dianchi\pilian\zong.xls"))
all_contents=np.array(all_contents)
print(len(all_contents))
np.random.seed(2022)
np.random.shuffle(all_contents)

def convert2oneHot(index,Lens):#onehot编码
    hot = np.zeros((Lens,))#np.zeros()返回来一个给定形状和类型的用0填充的数组；生成shape维度大小的全0数组。
    hot[int(index)-1] = 1
    return(hot)

def xs_gen(batch_size=Batch_size,train=True):

    if train:
        img_list = all_contents[:Lens]
        steps = math.ceil(len(img_list) / batch_size)    # 确定每轮有多少个batch
    else:
        img_list = all_contents[Lens:]
        steps = math.ceil(len(img_list) / batch_size)    # 确定每轮有多少个batch

    # img_list = pd.read_csv(path)
    # img_list = np.array(img_list)#np.array(x,dtype)：将x转化为一个类型为type的数组
    # print(len(img_list))
    while True:
        for i in range(steps):
            batch_list = img_list[i * batch_size: i * batch_size + batch_size]
            batch_x = np.array([file for file in batch_list[:,1:-1]])
            # print("label",len(batch_list[1,1:-1]))
            batch_y = np.array([convert2oneHot(label,2) for label in batch_list[:,-1]])
            # print(batch_x, batch_y)
            yield batch_x, batch_y
# X_test, y_test= xs_gen(train=False)
# X_train, X_test, y_train, y_test = train_test_split(dataSet, ansSet, test_size = 0.1, random_state=2)



#把标签转成oneHot



TIME_PERIODS = 651
def build_model (classnum=2):

    xin=tf.keras.layers.Input(shape=(TIME_PERIODS,1),dtype=tf.float32)

    x= Conv1D(16, 3, strides=2, input_shape=(TIME_PERIODS, 1))(xin)
    x= BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling1D(2)(x)

    x = Conv1D(64, 4, strides=2, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(64, 4, strides=2, padding="same")(x)
    x = BatchNormalization()(x)
    x= Activation('relu')(x)
    x = MaxPooling1D(2)(x)

    x = Conv1D(256, 4, strides=2, padding="same")(x)
    x= BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(256, 4, strides=2, padding="same")(x)
    x= BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling1D(2)(x)

    x = Conv1D(512, 2, strides=1, padding="same")(x)
    x= BatchNormalization()(x)
    x= Activation('relu')(x)
    x = Conv1D(512, 2, strides=1, padding="same")(x)
    x= BatchNormalization()(x)
    x= Activation('relu')(x)
    x = MaxPooling1D(2)(x)

    """model.add(Flatten())
    model.add(Dropout(0.3))
    model.add(Dense(256, activation='relu'))"""
    x = GlobalAveragePooling1D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.3)(x)
    x= Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    x= Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(classnum, activation='softmax')(x)

    model = tf.keras.Model(xin, x)

    return model

Train = True

if __name__ == "__main__":
    if Train == True:
        train_iter = xs_gen()
        val_iter = xs_gen(train=False)

        ckpt = ModelCheckpoint(
            filepath=r'D:\dianchi\pilian\gp6.h5',
            monitor='val_loss', save_best_only=True,verbose=1)

        model = build_model()
        opt = Adam(0.0001)
        model.compile(loss='binary_crossentropy',
                    optimizer=opt, metrics=['accuracy'])
        #print(model.summary())

        model.fit_generator(
            generator=train_iter,
            steps_per_epoch=Lens//Batch_size,
            epochs=100,
            initial_epoch=0,
            validation_data = val_iter,
            validation_steps = (Long - Lens)//Batch_size,
            callbacks=[ckpt],
            )
        SVG(model_to_dot(model).create(prog='dot',format='svg'))
        plot_model(model,to_file="7651234jl.png",show_shapes=True)       
        model.save(r"D:\dianchi\pilian\gp6.h5")
        model.save_weights(r"D:\dianchi\pilian\finishModelgp6.h5")


210


C:\Users\33287\anaconda3\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
12/12 [==============================] - 5s 77ms/step - loss: 0.7226 - accuracy: 0.6302
Epoch 2/100
12/12 [==============================] - 1s 79ms/step - loss: 0.6071 - accuracy: 0.7173
Epoch 3/100
12/12 [==============================] - 1s 75ms/step - loss: 0.5555 - accuracy: 0.7173
Epoch 4/100
12/12 [==============================] - 1s 76ms/step - loss: 0.4783 - accuracy: 0.8115
Epoch 5/100
12/12 [==============================] - 1s 84ms/step - loss: 0.4876 - accuracy: 0.7958
Epoch 6/100
12/12 [==============================] - 1s 94ms/step - loss: 0.4430 - accuracy: 0.8168
Epoch 7/100
12/12 [==============================] - 1s 79ms/step - loss: 0.3755 - accuracy: 0.8429
Epoch 8/100
12/12 [==============================] - 1s 119ms/step - loss: 0.3366 - accuracy: 0.8534
Epoch 9/100
12/12 [==============================] - 1s 92ms/step - loss: 0.3262 - accuracy: 0.8639
Epoch 10/100
12/12 [==============================] - 1s 93ms/step - loss: 0.2260 - accuracy: 0.911

12/12 [==============================] - 1s 73ms/step - loss: 1.9999e-04 - accuracy: 1.0000
Epoch 91/100
12/12 [==============================] - 1s 69ms/step - loss: 2.1462e-04 - accuracy: 1.0000
Epoch 92/100
12/12 [==============================] - 1s 66ms/step - loss: 3.5619e-04 - accuracy: 1.0000
Epoch 93/100
12/12 [==============================] - 1s 69ms/step - loss: 3.5183e-04 - accuracy: 1.0000
Epoch 94/100
12/12 [==============================] - 1s 108ms/step - loss: 1.4085e-04 - accuracy: 1.0000
Epoch 95/100
12/12 [==============================] - 1s 79ms/step - loss: 1.8991e-04 - accuracy: 1.0000
Epoch 96/100
12/12 [==============================] - 1s 72ms/step - loss: 4.6678e-04 - accuracy: 1.0000
Epoch 97/100
12/12 [==============================] - 1s 76ms/step - loss: 1.9585e-04 - accuracy: 1.0000
Epoch 98/100
12/12 [==============================] - 1s 85ms/step - loss: 4.5018e-04 - accuracy: 1.0000
Epoch 99/100
12/12 [==============================] - 1s 69ms/step 

In [98]:
#-*- coding:utf-8 -*-
import numpy as np
import math

import pandas as pd
import tensorflow as tf
#tf.compat.v1.disable_eager_execution()
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import *
import xlrd
from tensorflow.keras.models import *
#超参数设置，Batch_size为训练时每次的数据量，总共迭代600次，训练数据每次共300数据量，由于网络输入规定为32个，所以1次迭代要进行10次。
"""
Batch_size = 16
Long = 300
Lens = int(300*0.2)
"""
Batch_size = 16
Long = 220

Lens = int(220*0.4)
TestLens = int(220 * 0.6)
def DataSet(MANIFEST_DIRS):
#加载数据集，将 列 转化为 行
    all_content = []
    worksheet = xlrd.open_workbook(MANIFEST_DIRS)
    sheet_names = worksheet.sheet_names()#获取所有sheet的名称，以列表方式显示

    sheet = worksheet.sheet_by_name(sheet_names[0])#通过sheet名称获取所需sheet对象
    cols = sheet.ncols#获取某sheet中的有效列数

    for ncol in range(cols):
        #all_content.append(sheet.col_values(ncol)[1:])#1改为0
        all_content.append(sheet.col_values(ncol)[0:])
    all_content = np.array(all_content)

    return all_content

all_contents=list(DataSet(MANIFEST_DIRS=r"D:\dianchi\pilian\zong.xls"))
# a = list(DataSet(MANIFEST_DIRS=r"D:\数据集\西交数据集最终版\2-数据集\2-分类方式（列）\Horizontal_1_0.xls"))
#all_contents = []
#for i in range(10):
#    all_contents += list(DataSet(MANIFEST_DIRS=r"D:\DeepLearning\Bearing-fault-detection\工况3数据集\Vertical_3_"+ str(i) +".xls"))
# all_contents=list(DataSet(MANIFEST_DIRS=r"D:\数据集\西交数据集最终版\2-数据集\2-分类方式（列）\Horizontal_1_0.xls"))+list(DataSet(MANIFEST_DIRS=r"D:\数据集\西交数据集最终版\2-数据集\2-分类方式（列）\Horizontal_1_1.xls"))+list(DataSet(MANIFEST_DIRS=r"D:\数据集\西交数据集最终版\2-数据集\2-分类方式（列）\Horizontal_1_2.xls"))+list(DataSet(MANIFEST_DIRS=r"D:\数据集\西交数据集最终版\2-数据集\2-分类方式（列）\Horizontal_1_3.xls"))+list(DataSet(MANIFEST_DIRS=r"D:\数据集\西交数据集最终版\2-数据集\2-分类方式（列）\Horizontal_1_4.xls"))+list(DataSet(MANIFEST_DIRS=r"D:\数据集\西交数据集最终版\2-数据集\2-分类方式（列）\Horizontal_1_5.xls"))+list(DataSet(MANIFEST_DIRS=r"D:\数据集\西交数据集最终版\2-数据集\2-分类方式（列）\Horizontal_1_6.xls"))+list(DataSet(MANIFEST_DIRS=r"D:\数据集\西交数据集最终版\2-数据集\2-分类方式（列）\Horizontal_1_7.xls"))+list(DataSet(MANIFEST_DIRS=r"D:\数据集\西交数据集最终版\2-数据集\2-分类方式（列）\Horizontal_1_8.xls"))+list(DataSet(MANIFEST_DIRS=r"D:\数据集\西交数据集最终版\2-数据集\2-分类方式（列）\Horizontal_1_9.xls"))
#print(all_contents)
all_contents=np.array(all_contents)
print(len(all_contents))

np.random.seed(2021)#np. random. seed() 函数用于生成指定随机数。
np.random.shuffle(all_contents)
# numpy.random. shuffle(x)。shuffle()是不能直接访问的，可以导入numpy.random模块，然后通过 numpy.random 静态对象调用该方法，shuffle直接在原来的数组上进行操作，改变原来数组的顺序，无返回值(是对列表x中的所有元素随机打乱顺序，若x不是列表，则报错)。

#主要作用用于与神经网络经softmax输出的预测值最大值对应的索引，相比较
def convert2oneHot(index,Lens):#onehot编码，假如有6个类别，0类别 对应的onehot编码为 000000,1 000010
    hot = np.zeros((Lens,))#np.zeros()返回来一个给定形状和类型的用0填充的数组；生成shape维度大小的全0数组。
    hot[int(index)-1] = 1
    #hot[int(index) ] = 1
    return(hot)

def xs_gen(batch_size=Batch_size,train=True):
    if train:
        img_list = all_contents[:Lens]
        steps = math.ceil(len(img_list) / batch_size)    # 确定每轮有多少个batch
    else:
        img_list = all_contents[Lens:]
        steps = math.ceil(len(img_list) / batch_size)    # 确定每轮有多少个batch
    # img_list = pd.read_csv(path)
    # img_list = np.array(img_list)#np.array(x,dtype)：将x转化为一个类型为type的数组
    # print(len(img_list))
    while True:
        for i in range(steps):
            batch_list = img_list[i * batch_size: i * batch_size + batch_size]
            batch_x = np.array([file for file in batch_list[:,1:-1]])
            # print("label",len(batch_list[1,1:-1]))
            #batch_y = np.array([convert2oneHot(label,6) for label in batch_list[:,-1]])#6改为10
            batch_y = np.array([convert2oneHot(label, 2) for label in batch_list[:, -1]])
            # print(batch_x, batch_y)
            yield batch_x, batch_y#yield关键字使生成器函数执行暂停，yield关键字后面的表达式的值返回给生成器的调用者。
# X_test, y_test= xs_gen(train=False)
# X_train, X_test, y_train, y_test = train_test_split(dataSet, ansSet, test_size = 0.1, random_state=2)



#把标签转成oneHot



TIME_PERIODS = 651#时间段
def build_model (classnum=2):

    xin=tf.keras.layers.Input(shape=(TIME_PERIODS,1),dtype=tf.float32)
    #tf.keras.layers.Input()输入层解析
    #shape：输入的形状，tuple类型。不含batch_size；tuple的元素可以为None类型数据，表示未知的或者说任意的，一般这里不用None
    #dtype：数据类型，在大多数时候，我们需要的数据类型为tf.float32，因为在精度满足的情况下，float32运算更快。
    x= Conv1D(16, 8, strides=2, input_shape=(TIME_PERIODS, 1))(xin)
    ##16-输出的维度；卷积核的空域或时域窗长度,卷积核的尺寸，卷积核的大小为8；stride-卷积步长；
    x= BatchNormalization()(x)
    #BatchNormalization批量归一化（BN）是神经网络的标准化方法/层通常BN神经网络输入被归一化[0,1]或[-1,1]范围，或者意味着均值为0和方差等于1。
    x = Activation('relu')(x)
    x = MaxPooling1D(2)(x)#MaxPooling1D：是在steps维度（也就是第二维）求最大值。但是限制每一步的池化的大小。
    # 比如，输入数据维度是[10, 4, 10]，池化层大小pooling_size=2，步长stride=1，那么经过MaxPooling(pooling_size=2, stride=1)后，输出数据维度是[10, 3, 10]。

    x = Conv1D(64, 4, strides=2, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(64, 4, strides=2, padding="same")(x)
    x = BatchNormalization()(x)
    x= Activation('relu')(x)
    x = MaxPooling1D(2)(x)

    x = Conv1D(256, 4, strides=2, padding="same")(x)
    x= BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(256, 4, strides=2, padding="same")(x)
    x= BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling1D(2)(x)

    x = Conv1D(512, 2, strides=1, padding="same")(x)
    x= BatchNormalization()(x)
    x= Activation('relu')(x)
    x = Conv1D(512, 2, strides=1, padding="same")(x)
    x= BatchNormalization()(x)
    x= Activation('relu')(x)
    x = MaxPooling1D(2)(x)

    """model.add(Flatten())
    model.add(Dropout(0.3))
    model.add(Dense(256, activation='relu'))"""
    x = GlobalAveragePooling1D()(x)#GlobalMaxPooling1D:在steps维度（也就是第二维）对整个数据求最大值。比如说输入数据维度是[10, 4, 10]，那么进过全局池化后，输出数据的维度则变成[10, 10]。
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.3)(x)#防过拟合
    x= Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    x= Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(classnum, activation='softmax')(x)

    model = tf.keras.Model(xin, x)

    return model

Train = True

from sklearn.ensemble import RandomForestClassifier


def random_gen(train=True):

    if train:
        img_list1 = all_contents[:Lens]
    else:
        img_list1 = all_contents[Lens:]
    while True:
        batch_x1 = np.array([file for file in img_list1[:, 1:-1]])
        #batch_y1 = np.array([label-1 for label in img_list1[:, -1]])
        batch_y1 = np.array([label  for label in img_list1[:, -1]])
        return batch_x1, batch_y1

if __name__ == "__main__":
    if Train == True:
        train_iter = xs_gen()
        val_iter = xs_gen(train=False)

        ckpt = ModelCheckpoint(
            filepath=r'D:\dianchi\pilian\jn.h5',
            monitor='val_loss', save_best_only=True,verbose=20)

        model = build_model()

        opt = Adam(0.0005)
        model.compile(loss='categorical_crossentropy',
                    optimizer=opt, metrics=['accuracy'])

#         print(model.summary())

        model.fit_generator(
            generator=train_iter,
            steps_per_epoch=Lens//Batch_size,
            epochs=100,
            initial_epoch=0,
            validation_data = val_iter,
            validation_steps = (Long - Lens)//Batch_size,
            callbacks=[ckpt],
            )
        model.save(r"D:\dianchi\pilian\zongfinishModel.h5")
        # model.save_weights(r"E:\pycharm2021\pycharm\PyCharm2021.1.1\project\Bearing-fault-detection\weight/finishModel11.h5")
    train_iter = xs_gen()
    test_iter = xs_gen(train=False)
    # print('test_iter的值为',test_iter)
    train, label = random_gen()
    # print('label的值为', label)
    testtrain, testlabel = random_gen(train=False)
    # print('testlabel的值为', testlabel)

    # 达到降维的目的，从原有的32767降到128维
    model = tf.keras.models.load_model(filepath=r"D:\dianchi\pilian\zongfinishModel.h5")
    visualization_model = Model(inputs=model.input, outputs=model.get_layer('dense_2').output)
    pres = visualization_model.predict_generator(generator=train_iter, steps=math.ceil(Lens / Batch_size), verbose=1)
    # ohpres = np.argmax(pres, axis=1)
    # print(pres)
    models = RandomForestClassifier(n_estimators=25,
                                    n_jobs=2, min_samples_split=2, random_state=1, )
    #
    # # Fit on training data
    models.fit(pres, label)
    # print(pres.shape)

    # Pre = models.predict(pres)
    # num = 0
    # for n in range(len(label)):
    #     if int(Pre[n])== int(label[n] ):
    #         num = num + 1
    # print("准确率：", num / len(label))

    test_pres = visualization_model.predict_generator(generator=test_iter, steps=math.ceil(TestLens / Batch_size),
                                                      verbose=1)
    Pre = models.predict(test_pres)
    print(Pre)
    print(testlabel)
    #
    num = 0
    # for n in range(len(testlabel)):
    for n in range(len(testlabel) - 1):
        if int(Pre[n]) == int(testlabel[n]):
            num = num + 1
    print("准确率：", num / len(testlabel))

210
Epoch 1/100
5/5 [==============================] - 3s 191ms/step - loss: 0.6899 - accuracy: 0.7500 - val_loss: 0.6849 - val_accuracy: 0.6967

Epoch 00001: val_loss improved from inf to 0.68494, saving model to D:\dianchi\pilian\jn.h5
Epoch 2/100
5/5 [==============================] - 0s 106ms/step - loss: 0.8151 - accuracy: 0.5694 - val_loss: 0.6872 - val_accuracy: 0.6967

Epoch 00002: val_loss did not improve from 0.68494
Epoch 3/100
5/5 [==============================] - 0s 89ms/step - loss: 0.5693 - accuracy: 0.7500 - val_loss: 0.6765 - val_accuracy: 0.6967

Epoch 00003: val_loss improved from 0.68494 to 0.67655, saving model to D:\dianchi\pilian\jn.h5
Epoch 4/100
5/5 [==============================] - 0s 81ms/step - loss: 0.4671 - accuracy: 0.7917 - val_loss: 0.6680 - val_accuracy: 0.6967

Epoch 00004: val_loss improved from 0.67655 to 0.66804, saving model to D:\dianchi\pilian\jn.h5
Epoch 5/100
5/5 [==============================] - 0s 93ms/step - loss: 0.5457 - accuracy: 0.73

5/5 [==============================] - 0s 95ms/step - loss: 0.1212 - accuracy: 0.9583 - val_loss: 1.4731 - val_accuracy: 0.6967

Epoch 00041: val_loss did not improve from 0.59241
Epoch 42/100
5/5 [==============================] - 0s 94ms/step - loss: 0.1061 - accuracy: 0.9444 - val_loss: 1.4865 - val_accuracy: 0.6967

Epoch 00042: val_loss did not improve from 0.59241
Epoch 43/100
5/5 [==============================] - 0s 94ms/step - loss: 0.1265 - accuracy: 0.9250 - val_loss: 1.4386 - val_accuracy: 0.6967

Epoch 00043: val_loss did not improve from 0.59241
Epoch 44/100
5/5 [==============================] - 0s 96ms/step - loss: 0.0685 - accuracy: 0.9861 - val_loss: 1.4411 - val_accuracy: 0.6967

Epoch 00044: val_loss did not improve from 0.59241
Epoch 45/100
5/5 [==============================] - 0s 84ms/step - loss: 0.1449 - accuracy: 0.9444 - val_loss: 1.3430 - val_accuracy: 0.6967

Epoch 00045: val_loss did not improve from 0.59241
Epoch 46/100
5/5 [==============================

ValueError: No such layer: dense_2.

In [106]:
#-*- coding:utf-8 -*-
import numpy as np
import math

import pandas as pd
import tensorflow as tf
#tf.compat.v1.disable_eager_execution()
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import *
import xlrd
from tensorflow.keras.models import *
#超参数设置，Batch_size为训练时每次的数据量，总共迭代600次，训练数据每次共300数据量，由于网络输入规定为32个，所以1次迭代要进行10次。
"""
Batch_size = 16
Long = 300
Lens = int(300*0.2)
"""
Batch_size = 16
Long = 220

Lens = int(220*0.4)
TestLens = int(220 * 0.6)
def DataSet(MANIFEST_DIRS):
#加载数据集，将 列 转化为 行
    all_content = []
    worksheet = xlrd.open_workbook(MANIFEST_DIRS)
    sheet_names = worksheet.sheet_names()#获取所有sheet的名称，以列表方式显示

    sheet = worksheet.sheet_by_name(sheet_names[0])#通过sheet名称获取所需sheet对象
    cols = sheet.ncols#获取某sheet中的有效列数

    for ncol in range(cols):
        #all_content.append(sheet.col_values(ncol)[1:])#1改为0
        all_content.append(sheet.col_values(ncol)[0:])
    all_content = np.array(all_content)

    return all_content

all_contents=list(DataSet(MANIFEST_DIRS=r"D:\dianchi\pilian\zong.xls"))
# a = list(DataSet(MANIFEST_DIRS=r"D:\数据集\西交数据集最终版\2-数据集\2-分类方式（列）\Horizontal_1_0.xls"))
#all_contents = []
#for i in range(10):
#    all_contents += list(DataSet(MANIFEST_DIRS=r"D:\DeepLearning\Bearing-fault-detection\工况3数据集\Vertical_3_"+ str(i) +".xls"))
# all_contents=list(DataSet(MANIFEST_DIRS=r"D:\数据集\西交数据集最终版\2-数据集\2-分类方式（列）\Horizontal_1_0.xls"))+list(DataSet(MANIFEST_DIRS=r"D:\数据集\西交数据集最终版\2-数据集\2-分类方式（列）\Horizontal_1_1.xls"))+list(DataSet(MANIFEST_DIRS=r"D:\数据集\西交数据集最终版\2-数据集\2-分类方式（列）\Horizontal_1_2.xls"))+list(DataSet(MANIFEST_DIRS=r"D:\数据集\西交数据集最终版\2-数据集\2-分类方式（列）\Horizontal_1_3.xls"))+list(DataSet(MANIFEST_DIRS=r"D:\数据集\西交数据集最终版\2-数据集\2-分类方式（列）\Horizontal_1_4.xls"))+list(DataSet(MANIFEST_DIRS=r"D:\数据集\西交数据集最终版\2-数据集\2-分类方式（列）\Horizontal_1_5.xls"))+list(DataSet(MANIFEST_DIRS=r"D:\数据集\西交数据集最终版\2-数据集\2-分类方式（列）\Horizontal_1_6.xls"))+list(DataSet(MANIFEST_DIRS=r"D:\数据集\西交数据集最终版\2-数据集\2-分类方式（列）\Horizontal_1_7.xls"))+list(DataSet(MANIFEST_DIRS=r"D:\数据集\西交数据集最终版\2-数据集\2-分类方式（列）\Horizontal_1_8.xls"))+list(DataSet(MANIFEST_DIRS=r"D:\数据集\西交数据集最终版\2-数据集\2-分类方式（列）\Horizontal_1_9.xls"))
#print(all_contents)
all_contents=np.array(all_contents)
print(len(all_contents))

np.random.seed(2021)#np. random. seed() 函数用于生成指定随机数。
np.random.shuffle(all_contents)
# numpy.random. shuffle(x)。shuffle()是不能直接访问的，可以导入numpy.random模块，然后通过 numpy.random 静态对象调用该方法，shuffle直接在原来的数组上进行操作，改变原来数组的顺序，无返回值(是对列表x中的所有元素随机打乱顺序，若x不是列表，则报错)。

#主要作用用于与神经网络经softmax输出的预测值最大值对应的索引，相比较
def convert2oneHot(index,Lens):#onehot编码，假如有6个类别，0类别 对应的onehot编码为 000000,1 000010
    hot = np.zeros((Lens,))#np.zeros()返回来一个给定形状和类型的用0填充的数组；生成shape维度大小的全0数组。
    hot[int(index)-1] = 1
    #hot[int(index) ] = 1
    return(hot)

def xs_gen(batch_size=Batch_size,train=True):
    if train:
        img_list = all_contents[:Lens]
        steps = math.ceil(len(img_list) / batch_size)    # 确定每轮有多少个batch
    else:
        img_list = all_contents[Lens:]
        steps = math.ceil(len(img_list) / batch_size)    # 确定每轮有多少个batch
    # img_list = pd.read_csv(path)
    # img_list = np.array(img_list)#np.array(x,dtype)：将x转化为一个类型为type的数组
    # print(len(img_list))
    while True:
        for i in range(steps):
            batch_list = img_list[i * batch_size: i * batch_size + batch_size]
            batch_x = np.array([file for file in batch_list[:,1:-1]])
            # print("label",len(batch_list[1,1:-1]))
            #batch_y = np.array([convert2oneHot(label,2) for label in batch_list[:,-1]])#6改为10
            batch_y = np.array([convert2oneHot(label, 2) for label in batch_list[:, -1]])
            # print(batch_x, batch_y)
            yield batch_x, batch_y#yield关键字使生成器函数执行暂停，yield关键字后面的表达式的值返回给生成器的调用者。
# X_test, y_test= xs_gen(train=False)
# X_train, X_test, y_train, y_test = train_test_split(dataSet, ansSet, test_size = 0.1, random_state=2)



#把标签转成oneHot



TIME_PERIODS = 651#时间段
def build_model (classnum=2):

    xin=tf.keras.layers.Input(shape=(TIME_PERIODS,1),dtype=tf.float32)
    #tf.keras.layers.Input()输入层解析
    #shape：输入的形状，tuple类型。不含batch_size；tuple的元素可以为None类型数据，表示未知的或者说任意的，一般这里不用None
    #dtype：数据类型，在大多数时候，我们需要的数据类型为tf.float32，因为在精度满足的情况下，float32运算更快。
    x= Conv1D(16, 8, strides=2, input_shape=(TIME_PERIODS, 1))(xin)
    ##16-输出的维度；卷积核的空域或时域窗长度,卷积核的尺寸，卷积核的大小为8；stride-卷积步长；
    x= BatchNormalization()(x)
    #BatchNormalization批量归一化（BN）是神经网络的标准化方法/层通常BN神经网络输入被归一化[0,1]或[-1,1]范围，或者意味着均值为0和方差等于1。
    x = Activation('relu')(x)
    x = MaxPooling1D(2)(x)#MaxPooling1D：是在steps维度（也就是第二维）求最大值。但是限制每一步的池化的大小。
    # 比如，输入数据维度是[10, 4, 10]，池化层大小pooling_size=2，步长stride=1，那么经过MaxPooling(pooling_size=2, stride=1)后，输出数据维度是[10, 3, 10]。

    x = Conv1D(64, 4, strides=2, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(64, 4, strides=2, padding="same")(x)
    x = BatchNormalization()(x)
    x= Activation('relu')(x)
    x = MaxPooling1D(2)(x)

    x = Conv1D(256, 4, strides=2, padding="same")(x)
    x= BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(256, 4, strides=2, padding="same")(x)
    x= BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling1D(2)(x)

    x = Conv1D(512, 2, strides=1, padding="same")(x)
    x= BatchNormalization()(x)
    x= Activation('relu')(x)
    x = Conv1D(512, 2, strides=1, padding="same")(x)
    x= BatchNormalization()(x)
    x= Activation('relu')(x)
    x = MaxPooling1D(2)(x)

    """model.add(Flatten())
    model.add(Dropout(0.3))
    model.add(Dense(256, activation='relu'))"""
    x = GlobalAveragePooling1D()(x)#GlobalMaxPooling1D:在steps维度（也就是第二维）对整个数据求最大值。比如说输入数据维度是[10, 4, 10]，那么进过全局池化后，输出数据的维度则变成[10, 10]。
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.3)(x)#防过拟合
    x= Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    x= Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(classnum, activation='softmax')(x)

    model = tf.keras.Model(xin, x)

    return model

Train = True

from sklearn.ensemble import RandomForestClassifier


def random_gen(train=True):

    if train:
        img_list1 = all_contents[:Lens]
    else:
        img_list1 = all_contents[Lens:]
    while True:
        batch_x1 = np.array([file for file in img_list1[:, 1:-1]])
        #batch_y1 = np.array([label-1 for label in img_list1[:, -1]])
        batch_y1 = np.array([label  for label in img_list1[:, -1]])
        return batch_x1, batch_y1

if __name__ == "__main__":
    if Train == True:
        train_iter = xs_gen()
        val_iter = xs_gen(train=False)

        ckpt = ModelCheckpoint(
            filepath=r'D:\dianchi\pilian\jn.h5',
            monitor='val_loss', save_best_only=True,verbose=20)

        model = build_model()

        opt = Adam(0.0005)
        model.compile(loss='categorical_crossentropy',
                    optimizer=opt, metrics=['accuracy'])

        #print(model.summary())

        model.fit_generator(
            generator=train_iter,
            steps_per_epoch=Lens//Batch_size,
            epochs=10,
            initial_epoch=0,
            validation_data = val_iter,
            validation_steps = (Long - Lens)//Batch_size,
            callbacks=[ckpt],
            )
        model.save(r"D:\dianchi\pilian\zongfinishModel.h5")
        # model.save_weights(r"E:\pycharm2021\pycharm\PyCharm2021.1.1\project\Bearing-fault-detection\weight/finishModel11.h5")
    train_iter = xs_gen()
    test_iter = xs_gen(train=False)
    # print('test_iter的值为',test_iter)
    train, label = random_gen()
    # print('label的值为', label)
    testtrain, testlabel = random_gen(train=False)
    # print('testlabel的值为', testlabel)

    # 达到降维的目的，从原有的32767降到128维
    model = tf.keras.models.load_model(filepath=r"D:\dianchi\pilian\zongfinishModel.h5")
    visualization_model = Model(inputs=model.input, outputs=model.get_layer(index=3).output)
    pres = visualization_model.predict_generator(generator=train_iter, steps=math.ceil(Lens / Batch_size), verbose=1)
    # ohpres = np.argmax(pres, axis=1)
    # print(pres)
    models = RandomForestClassifier(n_estimators=25,
                                    n_jobs=2, min_samples_split=2, random_state=1, )
    #
    # # Fit on training data
    models.fit(pres, label)
    # print(pres.shape)

    # Pre = models.predict(pres)
    # num = 0
    # for n in range(len(label)):
    #     if int(Pre[n])== int(label[n] ):
    #         num = num + 1
    # print("准确率：", num / len(label))

    test_pres = visualization_model.predict_generator(generator=test_iter, steps=math.ceil(TestLens / Batch_size),
                                                      verbose=1)
    Pre = models.predict(test_pres)
    print(Pre)
    print(testlabel)
    #
    num = 0
    # for n in range(len(testlabel)):
    for n in range(len(testlabel) - 1):
        if int(Pre[n]) == int(testlabel[n]):
            num = num + 1
    print("准确率：", num / len(testlabel))

210
Epoch 1/10
5/5 [==============================] - 3s 207ms/step - loss: 0.8570 - accuracy: 0.6500 - val_loss: 0.6757 - val_accuracy: 0.6967

Epoch 00001: val_loss improved from inf to 0.67570, saving model to D:\dianchi\pilian\jn.h5
Epoch 2/10
5/5 [==============================] - 0s 96ms/step - loss: 0.5654 - accuracy: 0.7778 - val_loss: 0.6878 - val_accuracy: 0.6967

Epoch 00002: val_loss did not improve from 0.67570
Epoch 3/10
5/5 [==============================] - 0s 93ms/step - loss: 0.6550 - accuracy: 0.7083 - val_loss: 0.6755 - val_accuracy: 0.6967

Epoch 00003: val_loss improved from 0.67570 to 0.67552, saving model to D:\dianchi\pilian\jn.h5
Epoch 4/10
5/5 [==============================] - 0s 98ms/step - loss: 0.4754 - accuracy: 0.7778 - val_loss: 0.6766 - val_accuracy: 0.6967

Epoch 00004: val_loss did not improve from 0.67552
Epoch 5/10
5/5 [==============================] - 1s 105ms/step - loss: 0.4647 - accuracy: 0.8472 - val_loss: 0.6756 - val_accuracy: 0.6967

Epoc

C:\Users\33287\anaconda3\lib\site-packages\keras\engine\training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


ValueError: Found array with dim 3. Estimator expected <= 2.